In [85]:
import pandas as pd
import nltk  
nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from wordcloud import WordCloud, STOPWORDS
from string import punctuation
from collections import Counter

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
train_df = pd.read_csv("../input/nlp-getting-started/train.csv")

print("The csv shape: ", str(train_df.shape))
train_df.head()


The csv shape:  (7613, 5)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [87]:
print(" Example text: ", str(train_df["text"][1]), "\n", "Target: ", str(train_df["target"][1]))

 Example text:  Forest fire near La Ronge Sask. Canada 
 Target:  1


In [88]:
total = train_df.isnull().sum().sort_values(ascending = False)
percent = (train_df.isnull().sum())/(train_df.isnull().count()).sort_values(ascending = False)

missing_data = pd.concat([total, percent], axis = 1, keys = ["total", "percent"])
missing_data


,total,percent
location,2533,0.332720
keyword,61,0.008013
target,0,0.000000
text,0,0.000000
id,0,0.000000


In [89]:
train_df = train_df.drop(["keyword", "location", "id"], axis = 1)
print("Keyword, Location, and id are all dropped successfully")



Keyword, Location, and id are all dropped successfully


In [90]:
phrase = "Hello,,?? This is my Tweet!!!. I also want you to read this tweet https://www.youtube.com and also note https://relentless.com"
punct_list = set(punctuation)


def remove_punct(text):
    
    new_text = "".join(ch for ch in text if ch not in punct_list)
    return new_text

def remove_stopwords(text):
    
    text_split = text.split(" ")
    text = [word for word in text_split if word not in STOPWORDS]
    return text

def remove_http(text_list):
    
    new_text = [word for word in text_list if word.find("http") == -1]
    return new_text
    


remove_http(remove_stopwords(remove_punct(phrase.lower())))

['hello', 'tweet', 'want', 'read', 'tweet', 'note']

In [91]:
def preprocess_text(text):
    
    text = remove_http(remove_stopwords(remove_punct(text.lower())))
    return text

preprocess_text(phrase)
train_df["text"] = train_df["text"].apply(lambda x: preprocess_text(x))

['hello', 'tweet', 'want', 'read', 'tweet', 'note']

In [100]:
#The new preprocessed data
train_df.head()

,text,target
0,"[deeds, reason, earthquake, may, allah, forgiv...",1
1,"[forest, fire, near, la, ronge, sask, canada]",1
2,"[residents, asked, shelter, place, notified, o...",1
3,"[13000, people, receive, wildfires, evacuation...",1
4,"[got, sent, photo, ruby, alaska, smoke, wildfi...",1


In [102]:
train_df["text"]

0       [deeds, reason, earthquake, may, allah, forgiv...
1           [forest, fire, near, la, ronge, sask, canada]
2       [residents, asked, shelter, place, notified, o...
3       [13000, people, receive, wildfires, evacuation...
4       [got, sent, photo, ruby, alaska, smoke, wildfi...
                              ...                        
7608    [two, giant, cranes, holding, bridge, collapse...
7609    [ariaahrary, thetawniest, control, wild, fires...
7610             [m194, 0104, utc5km, s, volcano, hawaii]
7611    [police, investigating, ebike, collided, car, ...
7612    [latest, homes, razed, northern, california, w...
Name: text, Length: 7613, dtype: object

In [104]:
##### Turn the text into numbers

freq = {}
for row in train_df["text"]:
    
    for word in row:
        if word in freq:
            freq[word] += 1
        else:
            freq[word] = 1

print(freq)

{'the': 7613, 'latest:': 7613, 'more': 7613, 'homes': 7613, 'razed': 7613, 'by': 7613, 'northern': 7613, 'california': 7613, 'wildfire': 7613, '-': 7613, 'abc': 7613, 'news': 7613, 'http://t.co/ymy4rskq3d': 7613}


## Now I have to create a generator where:
* input: define length of padded array that contains the tweet
* label: whether that tweet needs action or not.